In [ ]:
Задание на практику: разработка микросервиса для распознавания объектов.

В рамках выполнения задания необходимо осуществить следующие этапы:

Обучение модели с использованием платформы Teachable Machine по адресу: https://teachablemachine.withgoogle.com/train.

Экспорт обученной модели.

Разработка API-интерфейса для обработки POST-запросов.

Реализация механизма приема файлов в качестве входных данных и возврата JSON-ответа, содержащего название распознанного объекта и уровень уверенности модели в идентификации.

Создание Dockerfile для контейнеризации приложения.

Размещение проекта на git.

In [ ]:
!pip install tensorflowjs flask pillow flask-cors numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.2
    Uninstalling packaging-24.2:
      Successfully uninstalled packaging-24.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.


In [ ]:
%%writefile app.py
from flask import Flask, request, jsonify
from werkzeug.utils import secure_filename
import tensorflow as tf
from tensorflow import keras
import numpy as np
from PIL import Image
import io
import os
from flask_cors import CORS
import logging

app = Flask(__name__)
CORS(app)

# Configuration
UPLOAD_FOLDER = '/tmp'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}
MODEL_PATH = 'model.h5'  # Update with your model path

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Model and labels
model = None
labels = ['Class1', 'Class2', 'Class3']  # Update with your class labels

def load_model():
    """Load the trained model"""
    global model
    try:
        if not os.path.exists(MODEL_PATH):
            raise FileNotFoundError(f"Model file not found at {MODEL_PATH}")

        logger.info(f"Loading model from {MODEL_PATH}")
        model = keras.models.load_model(MODEL_PATH)
        logger.info("Model loaded successfully")

        # Test model with dummy input
        try:
            dummy_input = np.zeros((1, 224, 224, 3))
            prediction = model.predict(dummy_input)
            logger.info(f"Model test prediction shape: {prediction.shape}")
        except Exception as e:
            logger.error(f"Model test failed: {str(e)}")
            raise

    except Exception as e:
        logger.error(f"Failed to load model: {str(e)}")
        raise

def allowed_file(filename):
    """Check if the file has an allowed extension"""
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

def preprocess_image(image_bytes):
    """Preprocess the uploaded image for model prediction"""
    try:
        logger.info("Preprocessing image")
        img = Image.open(io.BytesIO(image_bytes))

        # Convert grayscale to RGB if needed
        if img.mode != 'RGB':
            img = img.convert('RGB')

        img = img.resize((224, 224))  # Should match model's expected input
        img_array = keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0)  # Create batch of size 1

        # Normalize if your model expects it
        img_array = img_array / 255.0

        logger.info(f"Image processed, shape: {img_array.shape}")
        return img_array

    except Exception as e:
        logger.error(f"Image processing failed: {str(e)}")
        raise

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'model_loaded': model is not None,
        'endpoints': {
            '/recognize': 'POST'
        }
    })

@app.route('/recognize', methods=['POST'])
def recognize():
    """Endpoint for image recognition"""
    try:
        logger.info("Recognition request received")

        # Check if the post request has the file part
        if 'image' not in request.files:
            logger.error("No image in request")
            return jsonify({'error': 'No image provided'}), 400

        file = request.files['image']

        if file.filename == '':
            logger.error("Empty filename")
            return jsonify({'error': 'No selected file'}), 400

        if not file or not allowed_file(file.filename):
            logger.error(f"Invalid file type: {file.filename}")
            return jsonify({'error': 'Invalid file type'}), 400

        try:
            image_bytes = file.read()
            logger.info(f"Image received, size: {len(image_bytes)} bytes")

            # Preprocess the image
            tensor = preprocess_image(image_bytes)

            # Make prediction
            logger.info("Making prediction")
            predictions = model.predict(tensor)
            score = tf.nn.softmax(predictions[0])

            # Get results
            class_idx = np.argmax(score)
            confidence = 100 * np.max(score)

            logger.info(f"Prediction result: {labels[class_idx]} with confidence {confidence:.2f}%")

            return jsonify({
                'object': labels[class_idx],
                'confidence': float(confidence),
                'status': 'success'
            })

        except Exception as e:
            logger.error(f"Recognition error: {str(e)}")
            return jsonify({
                'error': 'Recognition failed',
                'details': str(e)
            }), 500

    except Exception as e:
        logger.error(f"Server error: {str(e)}")
        return jsonify({
            'error': 'Internal server error',
            'details': str(e)
        }), 500

if __name__ == '__main__':
    try:
        load_model()
        logger.info("Starting Flask server")
        app.run(host='0.0.0.0', port=3000, debug=True)
    except Exception as e:
        logger.error(f"Failed to start server: {str(e)}")

Overwriting app.py


In [ ]:
from google.colab import files
uploaded = files.upload()  # Загрузите ваш model.h5

Saving model.h5 to model.h5


In [ ]:
!python app.py

2025-07-11 15:17:09.963978: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752247030.001659    1369 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752247030.013474    1369 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-11 15:17:10.049384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/keras/src/ops/operation.py", line 234, in 

In [ ]:
!pip install pyngrok --upgrade
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar xvzf ngrok-v3-stable-linux-amd64.tgz
!chmod +x ngrok

from pyngrok import ngrok
ngrok.set_auth_token("2zVCk83hNI2ljf3YOciXDGAQ7Xc_4bo5TimrK4JmMahiSRBxf")  # Замените на ваш токен

--2025-07-11 15:23:11--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 75.2.60.68, 99.83.220.108, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|75.2.60.68|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9254557 (8.8M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   8.83M  --.-KB/s    in 0.05s   

2025-07-11 15:23:11 (170 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [9254557/9254557]

ngrok


In [1]:
from flask import Flask
import threading

app = Flask(__name__)

@app.route('/')
def hello():
    return "Микросервис распознавания объектов работает!"

def run_flask():
    app.run(port=3000)

threading.Thread(target=run_flask).start()

 * Serving Flask app '__main__'


In [2]:
!pip install pyngrok --upgrade
from pyngrok import ngrok

# 1. Установите свой токен (получите на https://dashboard.ngrok.com/get-started/your-authtoken)
ngrok.set_auth_token("2zVCk83hNI2ljf3YOciXDGAQ7Xc_4bo5TimrK4JmMahiSRBxf")

# 2. Создайте туннель с явным указанием протокола (важно для ngrok v3+)
public_url = ngrok.connect("http://localhost:3000", bind_tls=True)
print("Ваш API доступен по адресу:", public_url)

Ваш API доступен по адресу: NgrokTunnel: "https://26b53476eb81.ngrok-free.app" -> "http://localhost:3000"


In [ ]:
from google.colab import files
from PIL import Image
import io

uploaded = files.upload()
filename = list(uploaded.keys())[0]
image_bytes = io.BytesIO(uploaded[filename])

Saving download (1).jpg to download (1) (1).jpg


In [3]:
%%writefile Dockerfile
# Use official Python image
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    libgl1-mesa-glx \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first to leverage Docker cache
COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

# Copy the rest of the application
COPY . .

# Download model during build (alternative: mount volume at runtime)
# ADD https://example.com/path/to/model.h5 /app/model.h5

# Expose port
EXPOSE 3000

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:3000/health || exit 1

# Run the application
CMD ["gunicorn", "--bind", "0.0.0.0:3000", "--workers", "4", "app:app"]

Writing Dockerfile


In [4]:
%%writefile requirements.txt
flask==2.3.2
tensorflow==2.12.0
pillow==9.5.0
numpy==1.24.3
gunicorn==20.1.0
flask-cors==3.0.10

Writing requirements.txt


In [13]:
!git config --global user.email "ibramk79@gmail.com"
!git config --global user.name "Ibrahim-20"

In [14]:
!git init

Reinitialized existing Git repository in /content/.git/


In [15]:
%%writefile .gitignore
# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
env/

# Virtual environment
venv/
ENV/

# IDE
.vscode/
.idea/

# Data files
*.h5
*.jpg
*.png
*.jpeg

# Logs
*.log
logs/

# Docker
docker-compose.yml
Dockerfile

Overwriting .gitignore


In [16]:
!git add .
!git commit -m "Initial commit: Object recognition microservice"

On branch master
nothing to commit, working tree clean
